# **Image Project Phase one: Lane Detection**
the goal is to write a software pipeline to detect the lane boundaries in a video from a front-facing camera on a car. it’s required to find and track the lane lines and the position of the car from the center of the lane.
As a bonus, track the radius of curvature of the road too.

Assume the camera is mounted at the center of the car, such that the lane center is the midpoint at the bottom of the image between the two lines you've detected.

The offset of the lane center from the center of the image (converted from pixels to meters) is your distance from the center of the lane.

# Step 0: Import Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2965504/45929032 bytes (6.5%)6373376/45929032 bytes (13.9%)9838592/45929032 bytes (21.4%)13418496/45929032 bytes (29.2%)16785408/45929032 bytes (36.5%)20103168/45929032 bytes (43.8%)23502848/45929032 bytes (51.2%)26845184/45929032 bytes (58.4%)30277632/45929032 bytes (65.9%)33734656/45929032 bytes (73.4%)37167104/45929032 bytes (80.9%)40583168/45929032 bytes (88.4%)

# Step 1: Prespective Transformation Function (Bird's Eye)

In [4]:
"""Transformation Function (Normal ---> Bird'sEye)"""
def bird_forward(img, img_size=(1280, 720), flags=cv2.INTER_LINEAR):
    #Coordinates of input image
    input = np.float32([(550, 460),     # top-left
                            (150, 720),     # bottom-left
                            (1200, 720),    # bottom-right
                            (770, 460)])    # top-right
    #Coordinates of Output Image
    output = np.float32([(100, 0),
                            (100, 720),
                            (1100, 720),
                            (1100, 0)])
    #Transformation Matrix Calculation (Normal --> Bird's Eye)
    Mat = cv2.getPerspectiveTransform(input, output)
    return cv2.warpPerspective(img, Mat, img_size, flags=flags)

"""Inverse Transformation Function (Bird'sEye ---> Normal)"""
def bird_backward(img, img_size=(1280, 720), flags=cv2.INTER_LINEAR):
    #Coordinates of input image
    input = np.float32([(550, 460),     # top-left
                            (150, 720),     # bottom-left
                            (1200, 720),    # bottom-right
                            (770, 460)])    # top-right
    #Coordinates of Output Image
    output = np.float32([(100, 0),
                            (100, 720),
                            (1100, 720),
                            (1100, 0)])
    #Inverse Transformation Matrix Calculation  (Bird's Eye --> Normal)
    Mat_inv = cv2.getPerspectiveTransform(input, output)
    return cv2.warpPerspective(img, Mat_inv, img_size, flags=flags)

# Function to be applied to every Video's Frame

In [8]:
def process_image(img):

    """Step One: Transform the prespective of image to birdeye"""
    bird_img = bird_forward(img)
